In [300]:
##Set up imports, and default settings for Knowledge Base ID and AWS region

In [543]:
import os
import dotenv
import boto3
import pandas as pd

region = "us-east-1"
kb_id = "ARSJMG9P6M"

In [544]:
##.env has keys for access and secret. these can be set up in the IAM page of AWS console

In [545]:
dotenv.load_dotenv()

True

In [546]:
access = os.environ["access"]
secret = os.environ["secret"]

In [547]:
##set up boto3 session and bedrock agent runtime using access keys.

In [548]:
boto3_session = boto3.session.Session(region_name="us-east-1",aws_access_key_id=access,aws_secret_access_key=secret)
bedrock_agent_runtime_client = boto3_session.client('bedrock-agent-runtime')

In [549]:
#default values for the bedrock LLM being used.

In [550]:
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'


In [551]:
#define a prompt template to use for the chatbot. The template can have all the necessary definitions. Test and refine this prompt template.

In [552]:
custom_prompt_template= '''
You are a travel planning assistant that takes these inputs:
1) Details of a user's trip.
2) Encyclopedaeic information about the city the user is travelling to.
3) Interests of the user.

You must create a detailed itinerary for the duration of the user's trip. You need to ensure that you account for rest periods, travel time, price, and time of travel.
Your recommendations must align with the interests of the user, and details of their travel.

Use data from the search results, along with your own supplemental knowledge to create a detailed itinerary.

Trip details: {trip_details}

Retrieved Information: {search_results}

user interests: {user_interests}

Your Response:
'''

In [553]:
#load the csv file.

In [591]:
travel_db = pd.read_csv("final_travel_dataset.csv")

In [592]:
travel_db.columns

Index(['Unnamed: 0', 'Date_of_journey', 'Airline', 'Flight_code', 'Class',
       'Source', 'Departure', 'Total_stops', 'Arrival', 'Destination',
       'Duration_in_hours', 'Fare', 'availability'],
      dtype='object')

In [593]:
travel_db

Unnamed: 0 Date_of_journey    Airline Flight_code     Class  \
0                0      2024-01-16   SpiceJet     SG-8169   Economy   
1                1      2024-01-16     Indigo     6E-2519   Economy   
2                2      2024-01-16   GO FIRST      G8-354   Economy   
3                3      2024-01-16   SpiceJet     SG-8709   Economy   
4                4      2024-01-16  Air India      AI-805   Economy   
...            ...             ...        ...         ...       ...   
452083      452083      2024-03-06    Vistara      UK-926  Business   
452084      452084      2024-03-06    Vistara      UK-918  Business   
452085      452085      2024-03-06    Vistara      UK-918  Business   
452086      452086      2024-03-06    Vistara      UK-946  Business   
452087      452087      2024-03-06    Vistara      UK-946  Business   

           Source     Departure Total_stops       Arrival Destination  \
0           Delhi    After 6 PM    non-stop    After 6 PM      Mumbai   
1           Delhi    After 6 PM    non-stop   Before 6 AM      Mumbai   
2           Delhi    After 6 PM    non-stop   Before 6 AM      Mumbai   
3           Delhi    After 6 PM    non-stop    After 6 PM      Mumbai   
4           Delhi    After 6 PM    non-stop    After 6 PM      Mumbai   
...           ...           ...         ...           ...         ...   
452083  Ahmedabad  6 AM - 12 PM      1-stop    After 6 PM     Chennai   
452084  Ahmedabad   Before 6 AM      1-stop  12 PM - 6 PM     Chennai   
452085  Ahmedabad   Before 6 AM      1-stop  12 PM - 6 PM     Chennai   
452086  Ahmedabad  6 AM - 12 PM      1-stop    After 6 PM     Chennai   
452087  Ahmedabad  6 AM - 12 PM      1-stop    After 6 PM     Chennai   

        Duration_in_hours     Fare  availability  
0                  2.0833   5335.0            10  
1                  2.3333   5899.0            31  
2                  2.1667   5801.0            40  
3                  2.0833   5794.0            24  
4                  2.1667   5955.0            32  
...                   ...      ...           ...  
452083            13.0833  65028.0            11  
452084            11.2500  69254.0            34  
452085            11.2500  69254.0            38  
452086            11.1667  72980.0            25  
452087            11.1667  72980.0             0  

[452088 rows x 13 columns]

In [594]:
##BASIC EDA

In [595]:
travel_db['Date_of_journey'] = pd.to_datetime(travel_db['Date_of_journey'])

In [596]:
travel_db['Fare'] = travel_db['Fare'].astype(float)

In [597]:
travel_db.drop(columns=["Unnamed: 0"], inplace=True)

In [598]:
travel_db["Date_of_journey"].unique()

<DatetimeArray>
['2024-01-16 00:00:00', '2024-01-17 00:00:00', '2024-01-18 00:00:00',
 '2024-01-19 00:00:00', '2024-01-20 00:00:00', '2024-01-21 00:00:00',
 '2024-01-22 00:00:00', '2024-01-23 00:00:00', '2024-01-24 00:00:00',
 '2024-01-25 00:00:00', '2024-01-26 00:00:00', '2024-01-27 00:00:00',
 '2024-01-28 00:00:00', '2024-01-29 00:00:00', '2024-01-30 00:00:00',
 '2024-01-31 00:00:00', '2024-02-01 00:00:00', '2024-02-02 00:00:00',
 '2024-02-03 00:00:00', '2024-02-04 00:00:00', '2024-02-05 00:00:00',
 '2024-02-06 00:00:00', '2024-02-07 00:00:00', '2024-02-08 00:00:00',
 '2024-02-09 00:00:00', '2024-02-10 00:00:00', '2024-02-11 00:00:00',
 '2024-02-12 00:00:00', '2024-02-13 00:00:00', '2024-02-14 00:00:00',
 '2024-02-15 00:00:00', '2024-02-16 00:00:00', '2024-02-17 00:00:00',
 '2024-02-18 00:00:00', '2024-02-19 00:00:00', '2024-02-20 00:00:00',
 '2024-02-21 00:00:00', '2024-02-22 00:00:00', '2024-02-23 00:00:00',
 '2024-02-24 00:00:00', '2024-02-25 00:00:00', '2024-02-26 00:00:00',
 '20

In [599]:
##all inputs from the user are defined here. The questions, format, and input can vary depending on the use case.
#2024-02-03

In [564]:
source = input("Where are you leaving from?")
destination = input("Where are you going?")
date_of_departure = pd.to_datetime(input("What is the date of departure? (YYYY-MM-DD)"))
time_of_departure = input("Time of Departure: 1. '6 AM - 12 PM', 2. '12 PM - 6 PM', 3. 'After 6 PM', 4. 'Before 6 AM'. Enter an option from 1-4")
duration_of_trip= input("How many days is your trip for?")
#date_of_return = input("What is the date of return? (YYYY-MM-DD)")
number_of_adults = int(input("How many adults are travelling?"))
number_of_children = int(input("How many children are travelling?"))
budget = float(input("What is your budget per person?"))

In [565]:
time_of_departure='1'
time_map = {
    '1': '6 AM - 12 PM',
    '2': '12 PM - 6 PM',
    '3': 'After 6 PM',
    '4': 'Before 6 AM'
}
time_of_departure = time_map[time_of_departure]

In [566]:
##filter the dataset to only show the flights that match the needs of the user.

In [567]:
subset_travel_db = travel_db[(travel_db["Source"]==source) & (travel_db["Destination"]==destination)]

In [568]:
subset_travel_db = subset_travel_db[subset_travel_db['Date_of_journey'] == date_of_departure]

In [569]:
subset_travel_db = subset_travel_db[subset_travel_db['Fare']<=budget]

In [570]:
subset_travel_db = subset_travel_db[subset_travel_db['Departure']==time_of_departure]

In [571]:
#mask to drop non unique rows
subset_travel_db = subset_travel_db[subset_travel_db.nunique(axis=1) != 1]

In [572]:
#use the retriever function to get the necessary text blobs from the knowledge base.

In [573]:
subset_travel_db = subset_travel_db.reset_index()

In [574]:
subset_travel_db["Date_of_journey"].unique()

<DatetimeArray>
['2024-02-01 00:00:00']
Length: 1, dtype: datetime64[ns]

In [575]:
subset_travel_db

index Date_of_journey    Airline Flight_code    Class     Source  \
0   169411      2024-02-01    AirAsia      I5-819  Economy  Bangalore   
1   169419      2024-02-01    AirAsia     I5-1567  Economy  Bangalore   
2   169420      2024-02-01    AirAsia      I5-819  Economy  Bangalore   
3   169422      2024-02-01    AirAsia     I5-1452  Economy  Bangalore   
4   169423      2024-02-01    AirAsia     I5-2472  Economy  Bangalore   
5   169438      2024-02-01    Vistara      UK-810  Economy  Bangalore   
6   169439      2024-02-01    Vistara      UK-816  Economy  Bangalore   
7   169442      2024-02-01    Vistara      UK-812  Economy  Bangalore   
8   169449      2024-02-01   GO FIRST      G8-721  Economy  Bangalore   
9   169459      2024-02-01     Indigo      6E-484  Economy  Bangalore   
10  169463      2024-02-01     Indigo      6E-537  Economy  Bangalore   
11  169466      2024-02-01  Air India      AI-505  Economy  Bangalore   
12  169467      2024-02-01  Air India      AI-507  Economy  Bangalore   
13  169470      2024-02-01   SpiceJet      SG-531  Economy  Bangalore   
14  169471      2024-02-01     Indigo      6E-871  Economy  Bangalore   
15  169476      2024-02-01   SpiceJet      SG-531  Economy  Bangalore   
16  169477      2024-02-01     Indigo     6E-2186  Economy  Bangalore   
17  169479      2024-02-01    AirAsia     I5-1528  Economy  Bangalore   
18  169480      2024-02-01    AirAsia      I5-741  Economy  Bangalore   

       Departure Total_stops       Arrival Destination  Duration_in_hours  \
0   6 AM - 12 PM      1-stop  12 PM - 6 PM       Delhi             5.4167   
1   6 AM - 12 PM      1-stop  12 PM - 6 PM       Delhi             9.5000   
2   6 AM - 12 PM      1-stop    After 6 PM       Delhi            10.6667   
3   6 AM - 12 PM      1-stop    After 6 PM       Delhi            11.9167   
4   6 AM - 12 PM      1-stop    After 6 PM       Delhi            12.6667   
5   6 AM - 12 PM    non-stop  6 AM - 12 PM       Delhi             2.6667   
6   6 AM - 12 PM    non-stop  12 PM - 6 PM       Delhi             2.6667   
7   6 AM - 12 PM    non-stop  12 PM - 6 PM       Delhi             2.9167   
8   6 AM - 12 PM      1-stop    After 6 PM       Delhi            15.3333   
9   6 AM - 12 PM      1-stop  12 PM - 6 PM       Delhi             6.8333   
10  6 AM - 12 PM      1-stop    After 6 PM       Delhi             8.4167   
11  6 AM - 12 PM    non-stop  12 PM - 6 PM       Delhi             2.8333   
12  6 AM - 12 PM    non-stop  12 PM - 6 PM       Delhi             3.0000   
13  6 AM - 12 PM      1-stop  6 AM - 12 PM       Delhi            21.8333   
14  6 AM - 12 PM    non-stop  12 PM - 6 PM       Delhi             2.8333   
15  6 AM - 12 PM      1-stop    After 6 PM       Delhi             9.4167   
16  6 AM - 12 PM    non-stop  6 AM - 12 PM       Delhi             2.9167   
17  6 AM - 12 PM    non-stop  6 AM - 12 PM       Delhi             2.8333   
18  6 AM - 12 PM    non-stop  12 PM - 6 PM       Delhi             2.8333   

      Fare  availability  
0   5424.0            22  
1   5424.0             2  
2   5424.0             0  
3   5424.0            25  
4   5424.0             0  
5   5763.0            31  
6   5763.0            44  
7   5763.0            23  
8   6093.0            36  
9   5659.0             2  
10  5659.0            43  
11  5763.0            47  
12  5763.0             0  
13  6250.0            11  
14  5974.0            49  
15  6436.0            30  
16  6184.0             0  
17  6603.0             7  
18  6603.0            26

In [576]:
row_id = input("Select the Flight Code you want to select")

In [577]:
filtered_subset_travel_db = subset_travel_db[subset_travel_db['Flight_code'] == row_id]

In [600]:
filtered_subset_travel_db

index Date_of_journey    Airline Flight_code    Class     Source  \
12  169467      2024-02-01  Air India      AI-507  Economy  Bangalore   

       Departure Total_stops       Arrival Destination  Duration_in_hours  \
12  6 AM - 12 PM    non-stop  12 PM - 6 PM       Delhi                3.0   

      Fare  availability  
12  5763.0             0

CHECK FOR FLIGHTS WITH STOPS IF AVAILABILITY == 0

In [513]:
allowed_map = {
    "After 6 PM": "6 AM - 12 PM",#next day
    "Before 6 AM": "6 AM - 12 PM",#same day
    "12 PM - 6 PM": "After 6 PM",#same day
    "6 AM - 12 PM": "12 PM - 6 PM",#same day
    
}

In [622]:
import pandas as pd
#suppress warnings
import warnings

warnings.filterwarnings("ignore")
itineraries = []
if (filtered_subset_travel_db["availability"]==0).any():
    
    travel_db = travel_db[(travel_db["Total_stops"]=="non-stop")& (travel_db["availability"]>number_of_adults+number_of_children)]
    leg1_df = travel_db[(travel_db["Source"]==source) & (travel_db["Destination"]!=destination)]
    leg1_df = leg1_df[leg1_df["Fare"]<(budget/1.7)]
    leg1_df=leg1_df[leg1_df["Date_of_journey"]==date_of_departure]
    leg1_df.reset_index(inplace=True)
    
    for index, first_leg in leg1_df.iterrows():
        stop_city = first_leg['Destination']
        arrival_time = first_leg['Arrival']
        
        possible_second_legs= travel_db[(travel_db["Source"] == stop_city)&
                                        (travel_db["Departure"]== arrival_time )&
                                        (travel_db["Destination"]==destination)&
                                        (travel_db["Date_of_journey"]==date_of_departure)]
        #if possible_second_legs has more than one row, take the row with the cheapest fare
        if len(possible_second_legs)>1:
            possible_second_legs = possible_second_legs[possible_second_legs['Fare'] == possible_second_legs['Fare'].min()]
        #if the length is still greater than one, take the one with the highest availability
        
        if len(possible_second_legs)>1:
            possible_second_legs = possible_second_legs[possible_second_legs['availability'] == possible_second_legs['availability'].max()]
        if len(possible_second_legs)>1:
            #take the first row
            possible_second_legs = possible_second_legs.iloc[0]
        
        try:
            itin = {
            "first_leg": first_leg.to_dict(),
            "second_leg": possible_second_legs.to_dict(),
            "total_fare": float(first_leg["Fare"])+float(possible_second_legs["Fare"].values)
            }
            itineraries.append(itin)
        except Exception as e:
            continue
        
try:
    print(itineraries[0])
except Exception as e:
    print("the selected flight is available, or there are no connecting flights available")
    

{'first_leg': {'index': 186595, 'Date_of_journey': Timestamp('2024-02-01 00:00:00'), 'Airline': 'GO FIRST', 'Flight_code': 'G8-395', 'Class': 'Economy', 'Source': 'Bangalore', 'Departure': '6 AM - 12 PM', 'Total_stops': 'non-stop', 'Arrival': '6 AM - 12 PM', 'Destination': 'Mumbai', 'Duration_in_hours': 1.6667, 'Fare': 2252.0, 'availability': 22}, 'second_leg': {'Date_of_journey': {89217: Timestamp('2024-02-01 00:00:00')}, 'Airline': {89217: 'SpiceJet'}, 'Flight_code': {89217: 'SG-8702'}, 'Class': {89217: 'Economy'}, 'Source': {89217: 'Mumbai'}, 'Departure': {89217: '6 AM - 12 PM'}, 'Total_stops': {89217: 'non-stop'}, 'Arrival': {89217: '12 PM - 6 PM'}, 'Destination': {89217: 'Delhi'}, 'Duration_in_hours': {89217: 2.25}, 'Fare': {89217: 5843.0}, 'availability': {89217: 27}}, 'total_fare': 8095.0}


In [625]:
sorted_itins = sorted(itineraries, key=lambda x: x["total_fare"])

In [626]:
sorted_itins[:10]

[{'first_leg': {'index': 201510,
   'Date_of_journey': Timestamp('2024-02-01 00:00:00'),
   'Airline': 'AkasaAir',
   'Flight_code': 'QP-1415',
   'Class': 'Economy',
   'Source': 'Bangalore',
   'Departure': '6 AM - 12 PM',
   'Total_stops': 'non-stop',
   'Arrival': '6 AM - 12 PM',
   'Destination': 'Hyderabad',
   'Duration_in_hours': 1.3333,
   'Fare': 2305.0,
   'availability': 36},
  'second_leg': {'Date_of_journey': {241419: Timestamp('2024-02-01 00:00:00')},
   'Airline': {241419: 'GO FIRST'},
   'Flight_code': {241419: 'G8-424'},
   'Class': {241419: 'Economy'},
   'Source': {241419: 'Hyderabad'},
   'Departure': {241419: '6 AM - 12 PM'},
   'Total_stops': {241419: 'non-stop'},
   'Arrival': {241419: '6 AM - 12 PM'},
   'Destination': {241419: 'Delhi'},
   'Duration_in_hours': {241419: 2.4167},
   'Fare': {241419: 5169.0},
   'availability': {241419: 37}},
  'total_fare': 7474.0},
 {'first_leg': {'index': 201511,
   'Date_of_journey': Timestamp('2024-02-01 00:00:00'),
   'Airl

In [623]:
len(itineraries)

64

In [388]:
interests = input("What are your group's interests? This will be used to plan your itinerary. Ex: shopping, history, art, etc.")

In [389]:
retrieval_response = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id,
    nextToken='string',
    
    retrievalQuery={
        #'text': destination+": culture, art, tourist spots, and places to visit."
        'text':destination+" "+ interests
    }
)

In [339]:
##combine all the text into a single variable that can be pushed to the prompt template.

In [390]:
knowledge = ""
reference = retrieval_response["retrievalResults"][0]['location']['s3Location']
for item in retrieval_response["retrievalResults"]:
    knowledge+= item["content"]["text"]
    knowledge+= "\n"

In [391]:
knowledge

'George maintains a collection of objects of the British era. The museum is managed by the Archaeological Survey of India and has in its possession, the first Flag of India hoisted at Fort St George after the declaration of India\'s Independence on 15 August 1947.\r Chennai is the base for Tamil cinema, nicknamed Kollywood, alluding to the neighbourhood of Kodambakkam where several film studios are located. The history of cinema in South India started in 1897 when a European exhibitor first screened a selection of silent short films at the Victoria Public Hall in the city. Swamikannu Vincent purchased a film projector and erected tents for screening films which became popular in the early 20th century. Keechaka Vadham, the first film in South India was produced in the city and released in 1917. Gemini and Vijaya Vauhini studios were established in the 1940s, amongst the largest and earliest in the country. Chennai hosts many major film studios, including AVM Productions, the oldest sur

In [ ]:
##define a variable that can be pushed into the prompt template.

In [392]:
if 'days' or "Days" not in duration_of_trip:
    duration_of_trip+=' days'

In [393]:
trip_details = {
    "destination":destination,
    "duration_of_trip": duration_of_trip,
    "adults": number_of_adults,
    "children": number_of_children
     
}

In [394]:
trip_details


{'destination': 'Chennai',
 'duration_of_trip': '4 days days',
 'adults': '2',
 'children': '0'}

In [345]:
##format the prompt template with the user inputs

In [395]:
final_prompt=custom_prompt_template.format(
    trip_details=trip_details,
    search_results=knowledge,
    user_interests=interests
    )

In [396]:
final_prompt

'\nYou are a travel planning assistant that takes these inputs:\n1) Details of a user\'s trip.\n2) Encyclopedaeic information about the city the user is travelling to.\n3) Interests of the user.\n\nYou must create a detailed itinerary for the duration of the user\'s trip. You need to ensure that you account for rest periods, travel time, price, and time of travel.\nYour recommendations must align with the interests of the user, and details of their travel.\n\nUse data from the search results, along with your own supplemental knowledge to create a detailed itinerary.\n\nTrip details: {\'destination\': \'Chennai\', \'duration_of_trip\': \'4 days days\', \'adults\': \'2\', \'children\': \'0\'}\n\nRetrieved Information: George maintains a collection of objects of the British era. The museum is managed by the Archaeological Survey of India and has in its possession, the first Flag of India hoisted at Fort St George after the declaration of India\'s Independence on 15 August 1947.\r Chennai 

In [35]:
#combine these search results into one paragraph, and push it into the prompt template. Format the prompt template, and send the retrieval query.

In [36]:
##define a function that takes in all the values, uses the retrieve_and_generate function of the bedrock knowledge base and generates the input based on the prompt template. Instead of this function, a BedrockChat object can be defined with which the prompt can be pushed to the bedrock model directly.

In [397]:
def retrieveAndGenerate(input, kbId, model_arn, sessionId):
    
    
    
    #print(input, kbId, model_arn, sessionId)
    if sessionId != "":
        return bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kbId,
                    'modelArn': model_arn
                },
                'promptTemplate': {
                    'textPromptTemplate': custom_prompt_template
                },
                'knowledgeBaseConfiguration':{
                    'promptTemplate': {
                    'textPromptTemplate': custom_prompt_template
                },
                }
            },
            sessionId=sessionId
        )
    else:
        return bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kbId,
                    'modelArn': model_arn
                }
            }
        )

In [398]:
query = ""
sessionId = ""
response = retrieveAndGenerate(final_prompt, kb_id, model_arn, sessionId)

In [197]:
##print output. needs to be formatted.

In [399]:
filtered_subset_travel_db

index  Unnamed: 0 Date_of_journey Airline Flight_code    Class     Source  \
2  222333      222333      2024-01-16  Indigo      6E-904  Economy  Bangalore   

      Departure Total_stops       Arrival Destination  Duration_in_hours  \
2  6 AM - 12 PM    non-stop  6 AM - 12 PM     Chennai             1.0833   

     Fare  
2  2475.0

In [400]:
response['output']['text']

"For a 4-day trip to Chennai with interests in food and museums, here is a suggested itinerary:\n\nDay 1:\n- Start your day early with a traditional South Indian breakfast like idli, dosa or vada at a local restaurant like Murugan Idli Shop or Ratna Cafe. These places serve food on banana leaves, an old custom that imparts a unique flavor (Source: 2).\n- Visit the Government Museum, one of the oldest museums in India, to see its vast collection of archaeological artifacts, sculptures, and bronzes (Source: 5).\n- Have lunch at Murugan Idli Shop or Ratna Cafe, savoring more of Chennai's delicious vegetarian fare.\n- In the afternoon, explore Fort St. George and its museum, which houses objects from the British era including the first flag of independent India (Source: 5). Day 2: \n- Start your day at the National Art Gallery, another of Chennai's oldest museums, to view its impressive collection of paintings and sculptures (Source: 5).\n- For lunch, indulge in some of Chennai's famous st